Capstone 3 - Data Wrangling

In [1]:
#Import pandas, matplotlib.pyplot, and seaborn 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]


In [16]:
### Load Dataset

file = '/Users/elliotgaynon/Desktop/CAP3/Capstone3/data/raw/Online Retail.xlsx'

data_raw = pd.read_excel(file)

In [18]:
### Confirm DataFrame is loaded
data_raw

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


**I. Verify Columns and Data Types**

In [20]:
data_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [22]:
data_raw.shape

(541909, 8)

We have 541909 rows and 8 columns

In [23]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


Categorical variables appear as objects, continuous variables appear as either integers or floats, and my InvoiceDate appears as a datetime object. This looks good.  Now I will check for missing values.

***II. Check for Missing Values + Data Cleaning***

In [25]:
### Checking DataFrame for missing values

data_raw.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

***I have 1,454 missing Description values and 135,080 missing values for CustomerID.***


For missing Description values, I will leave them as is since they won't be used for RFM features or my target consrtuction. Dropping these rows would gain me nothing and filling them would be unnecessary.

On the other hand, CustomerID is what I'll be aggregating and computing RFM metrics with.  If I have no data here, I won't be able to perform my analysis.

In [26]:
### Dropping Missing CustomerID

data_raw = data_raw.dropna(subset = ['CustomerID'])


In [28]:
###Confirm missing values are gone in CustomerID
data_raw.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

It looks like when I got rid of missing CustomerID values, my missing description values appeared to disappear as well.  I want to see if any missing descriptions existed only among rows with valid CustomerIDs

In [29]:
data_raw[data_raw['CustomerID'].notna()]['Description'].isna().sum()

np.int64(0)

Zero descriptions existed among valid CustomerIDs. This confirms that all missing descriptions were tied to missing CustomerIDs.  Thus, no usable customer data was lost. Now all rows with missing customerID data, as well as missing product descriptions, are gone.

***III. Checking for canceled invoices, negative or zero prices and Negative quantities***

The online retail dataset website states "If the Invoice code starts with letter 'c', it indicates a cancellation."  A cancelled invoice means there's no revenue.  Thus, in order to calculate my target variable, Monetary Value, and Frequency, I must remove these invoices.

In [34]:
### Inspect InvoiceNo

num_canceled = data_raw['InvoiceNo'].astype(str).str.startswith('C').sum()
num_canceled

np.int64(8905)

There are 8905 Invoices that start with the letter 'C'.  They must be removed.

Since Revenue = Quantity x UnitPrice negative quantities of items or unit prices must be removed.  We must only have revenue generating sales in the model of customer value.  Thus, these items must be removed as well since we are trying to predict future sales.

In [40]:
### Check for Non-Positive Quantities

num_neg_qty =(data_raw['Quantity'] <= 0).sum()
num_neg_qty

np.int64(8905)

There are 8905 rows who's quantities are less than or equal to zero.  This is the same number as our canceled orders!

In [42]:
### Check for Non-Positive Prices

num_neg_prc = (data_raw['UnitPrice'] <= 0).sum()
num_neg_prc

np.int64(40)

There are 40 rows where the price is less than or equal to zero. 

In [43]:
### Overlap Check

data_raw[
    data_raw['InvoiceNo'].astype(str).str.startswith('C') |
    (data_raw['Quantity'] <= 0) |
    (data_raw['UnitPrice'] <= 0)
].shape[0]

8945

8945 rows must be removed in total

In [44]:
### Revenue Sanity check

(data_raw['Quantity'] * data_raw['UnitPrice']).describe()

count    406829.000000
mean         20.401854
std         427.591718
min     -168469.600000
25%           4.200000
50%          11.100000
75%          19.500000
max      168469.600000
dtype: float64

My minimum value is a very large negative value. This confirms quantity and unit price must be cleaned out.

***IV. Removing canceled invoices, non-positive quantity/price***

In [49]:
### remove canceled invoices

before = data_raw.shape[0]

mask_canceled = data_raw['InvoiceNo'].astype(str).str.startswith('C')
data_raw = data_raw[~mask_canceled]

after = data_raw.shape[0]
print(f"Dropped {before - after} canceled invoice rows")
print(f"Remaining rows: {after}")

Dropped 8905 canceled invoice rows
Remaining rows: 397924


In [50]:
### Remove Non-Positive Quantities

before = data_raw.shape[0]

data_raw = data_raw[data_raw['Quantity'] > 0]

after = data_raw.shape[0]
print(f"Dropped {before - after} rows with non-positive Quantity")
print(f"Remaining rows: {after}")

Dropped 0 rows with non-positive Quantity
Remaining rows: 397924


This makes sense since non-positive quantities was the same as canceled transactions.

In [51]:
### Remove Non-Positive Pricing

before = data_raw.shape[0]

data_raw = data_raw[data_raw['UnitPrice'] > 0]

after = data_raw.shape[0]
print(f"Dropped {before - after} rows with non-positive Quantity")
print(f"Remaining rows: {after}")

Dropped 40 rows with non-positive Quantity
Remaining rows: 397884


In [52]:
### Final Confirmation
print(f"Final cleaned rows: {data_raw.shape[0]}")

Final cleaned rows: 397884


In [53]:
### Checking for any missing values

data_raw.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

No Missing values.  Can move on.

***V. Creation of Revenue Column***

Before moving on, I want to double confirm UnitPrice and Quantity are greater than 0.

In [56]:
### Sanity Check

(data_raw['Quantity'] <= 0).sum(), (data_raw['UnitPrice']<=0).sum()

(np.int64(0), np.int64(0))

All values of quantity and unit price are where we want them.  

In [58]:
### Create the Revenue Column - Revenue = Quantity x UnitPrice

data_raw['Revenue'] = data_raw['Quantity'] * data_raw['UnitPrice']

In [59]:
### Description of new Revenue Column

data_raw['Revenue'].describe()

count    397884.000000
mean         22.397000
std         309.071041
min           0.001000
25%           4.680000
50%          11.800000
75%          19.800000
max      168469.600000
Name: Revenue, dtype: float64

Minimum is greater than 0.  No huge negatives. 

In [60]:
### Quick Spot check 

data_raw[['Quantity', 'UnitPrice', 'Revenue']].head()

,Quantity,UnitPrice,Revenue
0,6,2.55,15.30
1,6,3.39,20.34
2,8,2.75,22.00
3,6,3.39,20.34
4,6,3.39,20.34


***VI. Data Handling and Cutoff***

For selecting the cutoff date, I must consider a minimum window of past transactions in order to calculate RFM(Recency, Frequency, and MonetaryValue).  Thus, cutoff date + 60 days must fit within the data set.  Also I want to choose a cutoff that isn't too close to the least transaction date.

In [65]:
### Inspect Dataset Dates - confirm datetime

data_raw['InvoiceDate'].info()

<class 'pandas.core.series.Series'>
Index: 397884 entries, 0 to 541908
Series name: InvoiceDate
Non-Null Count   Dtype         
--------------   -----         
397884 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 6.1 MB


In [66]:
### check first and last transactions

print("Earliest transaction:", data_raw['InvoiceDate'].min())
print("Latest transaction:", data_raw['InvoiceDate'].max())

Earliest transaction: 2010-12-01 08:26:00
Latest transaction: 2011-12-09 12:50:00


The transactions occured from December 1st, 2010 until December 9th, 2011.  A little over a year.

The latest safe cutoff = the latest transaction minus 60 days.

In [69]:
### Choose Cutoff Date

import datetime

last_date = data_raw['InvoiceDate'].max()

cutoff_date = last_date - datetime.timedelta(days=60)

print("Last Date:", last_date)
print("Cutoff date:" , cutoff_date)

Last Date: 2011-12-09 12:50:00
Cutoff date: 2011-10-10 12:50:00


In [70]:
### Check Transaction Volume Around cutoff date 

transactions_per_day = data_raw.groupby('InvoiceDate').size()
transactions_per_day[cutoff_date - pd.Timedelta(days=10) : cutoff_date + pd.Timedelta(days=10)]

InvoiceDate
2011-09-30 12:58:00    14
2011-09-30 13:06:00    17
2011-09-30 13:09:00    33
2011-09-30 13:15:00    23
2011-09-30 13:21:00     5
                       ..
2011-10-20 12:31:00    11
2011-10-20 12:34:00    40
2011-10-20 12:40:00     1
2011-10-20 12:41:00     6
2011-10-20 12:43:00    95
Length: 1157, dtype: int64

Cutoff date appears to fall in a reasonably active period.  No major holidays within this range.
Thus, 

Features = all transactions ≤ 2011-10-10

Target = sum of revenue for each customer 2011-10-11 → 2011-12-09 (60-day window)

In [74]:
### Convert CustomerID from float to int

data_raw['CustomerID'] = data_raw['CustomerID'].astype(int)

In [75]:
### Sanity Check

data_raw['CustomerID'].dtype

dtype('int64')

In [76]:
### saving cleaned data_set as csv

data_raw.to_csv("cleaned_transactions.csv", index=False)